# CT5170: Principles of ML - Assignment 2
## Course code: 1MAO3
### Participants (name: id): [ Daniel Verdejo: 22240224, Tom Cronin: < id >]


### TODO: DELETE THIS - ITS SOME BASIC INFORMATION AS A REFERENCE 

## Definition of the perceptron
The perceptron is a linear-model binary classifier with a simple input—output relationship as depicted in Figure 2-3, which shows we're summing n number of inputs times their associated weights and then sending this "net input" to a step function with a defined threshold. Typically with perceptrons, this is a Heaviside step function with a threshold value of 0.5. This function will output a real-valued single binary value (0 or a 1), depending on the input.

![perceptron](assets/perceptronFig.png?raw=true)

We can model the decision boundary and the classification output in the Heaviside
step function equation, as follows:

![f of x](assets/fx.png)

To produce the net input to the activation function (here, the Heaviside step function) we take the dot product of the input and the connection weights. We see this
summation in the left half of Figure 2-3 as the input to the summation function.
Table 2-1 provides an explanation of how the summation function is performed as
well as notes about the parameters involved in the summation function.


![params](assets/params.png)

The output of the step function (activation function) is the output for the perceptron and gives us a classification of the input values. If the bias value is negative, it forces the learned weights sum to be a much greater value to get a 1 classification output. The bias term in this capacity moves the decision boundary around for the model. Input values do not affect the bias term, but the bias term is learned through the perceptron learning algorithm.

## The perceptron learning algorithm
The perceptron learning algorithm changes the weights in the perceptron model until all input records are all correctly classified. The algorithm will not terminate if the learning input is not linearly separable. A linearly separable dataset is one for which we can find the values of a hyperplane that will cleanly divide the two classes of the dataset.

The perceptron learning algorithm initializes the weight vector with small random values or 0.0s at the beginning of training. The perceptron learning algorithm takes each input record, as we can see in Figure 2-3, and computes the output classification to check against the actual classification label. To produce the classification, the columns (features) are matched up to weights where n is the number of dimensions in both our input and weights. The first input value is the bias input, which is always 1.0 because we don't affect the bias input. The first weight is our bias term in this diagram. The dot product of the input vector and the weight vector gives us the input to our activation function, as we've previously discussed.

If the classification is correct, no weight changes are made. If the classification is incorrect, the weights are adjusted accordingly. Weights are updated between individual training examples in an "online learning" fashion. This loop continues until all of the input examples are correctly classified. If the dataset is not linearly separable, the training algorithm will not terminate. Figure 2-4 demonstrates a dataset that is not linearly separable, the XOR logic function.

![xor](assets/xor.png)

In [8]:
import pandas as pd
import numpy as np

import doctest


In [9]:
# Tom Cronin
def read_data_return_dataframe(PathToFile):
    """
    Should return a dataframe of the test file
    >>> type(read_data_return_dataframe('testdata.txt'))
    <class 'pandas.core.frame.DataFrame'>

     Should have correct data in the dataframe.
     There should be 20 entrys in the dataframe excluding of the column names
    >>> read_data_return_dataframe('testdata.txt').size
    20
    """
    return pd.read_table(PathToFile) # reads txt file and converts it to a pandas dataframe

In [10]:
doctest.run_docstring_examples(read_data_return_dataframe, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    type(read_data_return_dataframe('testdata.txt'))
Expecting:
    <class 'pandas.core.frame.DataFrame'>
ok
Trying:
    read_data_return_dataframe('testdata.txt').size
Expecting:
    20
ok


In [19]:
# Daniel Verdejo - split into labels and attributes
def split_df_labels_attributes(df):
    """ Split the dataframe into two by labels and attributes

        Keyword arguments:
        df -- A pandas dataframe type containing labels and attributes
        label_col_name -- A string which contains the name of the label column. 

        Returns:
        tuple -- (label: pd.DataFrame, attributes: pd.DataFrame)

        Should output only the Label column
        >>> labels, attrs = split_df_labels_attributes(read_data_return_dataframe('testdata.txt'))
        >>> labels
                  Label
                0  no   
                1  no

        Should not contain the Label column
        >>> labels, attrs = split_df_labels_attributes(read_data_return_dataframe('testdata-alt.txt'))
        >>> 'label' in attrs
        False
        
        >>> attrs.columns
        Index(['year', 'temp', 'humidity', 'rainfall', 'drought_code', 'buildup_index',
          'day', 'month', 'wind_speed'],
        dtype='object')
      """
    return (df.iloc[:,0:1], df.iloc[:,1:])  # (for every row take columns upto index 1 exclusive, for every row take every column from 1 onwards inclusive)

In [20]:
doctest.run_docstring_examples(split_df_labels_attributes, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    labels, attrs = split_df_labels_attributes(read_data_return_dataframe('testdata.txt'))
Expecting nothing
ok
Trying:
    labels
Expecting:
              Label
            0  no   
            1  no
ok
Trying:
    labels, attrs = split_df_labels_attributes(read_data_return_dataframe('testdata-alt.txt'))
Expecting nothing
ok
Trying:
    'label' in attrs
Expecting:
    False
ok
Trying:
    attrs.columns
Expecting:
    Index(['year', 'temp', 'humidity', 'rainfall', 'drought_code', 'buildup_index',
      'day', 'month', 'wind_speed'],
    dtype='object')
ok


In [159]:
# Daniel Verdejo - train, test splilt dataframe
def split_df_to_train_test_dfs(df):
    """ Splits a single dataframe into 2 dataframes
    
    Arguments:
    df -- A pandas Dataframe to be split into 2

    Returns:
    tuple -- (df_train: pandas.Dataframe, df_test: pandas.Dataframe)
    
    Should split a single dataframe into 2 unique dataframes
    >>> df = read_data_return_dataframe('wildfires.txt')
    >>> train, test = split_df_to_train_test_dfs(df)
    >>> train.index.equals(tr.index)
    False

    Length of train and test dataframes should equal the length of the orignal
    >>> df = read_data_return_dataframe('wildfires.txt')
    >>> train, test = split_df_to_train_test_dfs(df)
    >>> len(train) + len(test) == len(df)
    True

    Should contain different values
    >>> df = read_data_return_dataframe('wildfires.txt')
    >>> train, test = split_df_to_train_test_dfs(df)
    >>> train.values != test.values
    True
    """
    train_frac = round(np.random.uniform(.6, .7), 2) # get a random float for our training fraction
    df_train = df.sample(frac = train_frac) # randomly sample a fraction of the dataframe between 60 & 70 % of its entirety
    return (df_train,  df.drop(df_train.index)) # return the training data and the test data

C:\Users\danny\AppData\Local\Temp\ipykernel_23668\4234067064.py:36: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  tr.values != te.values # te.index


True

In [160]:
doctest.run_docstring_examples(split_df_to_train_test_dfs, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    df = read_data_return_dataframe('wildfires.txt')
Expecting nothing
ok
Trying:
    train, test = split_df_to_train_test_dfs(df)
Expecting nothing
ok
Trying:
    train.index.equals(tr.index)
Expecting:
    False
ok
Trying:
    df = read_data_return_dataframe('wildfires.txt')
Expecting nothing
ok
Trying:
    train, test = split_df_to_train_test_dfs(df)
Expecting nothing
ok
Trying:
    len(train) + len(test) == len(df)
Expecting:
    True
ok
Trying:
    df = read_data_return_dataframe('wildfires.txt')
Expecting nothing
ok
Trying:
    train, test = split_df_to_train_test_dfs(df)
Expecting nothing
ok
Trying:
    train.values != test.values
Expecting:
    True
ok


<doctest NoName[8]>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  train.values != test.values


In [14]:
# Daniel Verdejo - initial perceptron based on description above
class Perceptron:

    def __activation_fn__(x):
        return np.where(x >= 0, 1, 0)

    def __init__(self, learn_rate, n_iters):
        self.lr = learn_rate
        self.n_iters = n_iters
        self.activation_fn = (lambda x: np.where(x >= 0, 1, 0)) # TODO: might need to revisit this
        self.weights = None
        self.bias = None

    def __str__():
        pass

    def fit(self, X, y=None):
        print(X)

        if(isinstance(X, pd.DataFrame)):
            X = X.to_numpy()[:,1:] # exclude the outcomes here will need to do this before 
        _, cnt_features = X.shape

        self.weights = np.random.rand(cnt_features)
        self.bias = 0

        for _ in range(self.n_iters):
            for i, x_i in enumerate(X):
                ln_out = np.dot(x_i, self.weights) + self.bias
                pred = self.activation_fn(ln_out)
                update = self.lr * (y[i] - pred)

                self.weights = self.weights + (update * x_i)
                self.bias += update
                
        return self

    def predict(self, X):
        print(X)
        ln_out = np.dot(X, self.weights) + self.bias
        pred = self.activation_fn(ln_out)
        print(pred)
        return pred


        

    def fit_predict(self):
        return self
    
    def evaluate(self):
        pass

# TODO: remove these when done testing! 
# X = read_data_return_dataframe("./wildfires.txt")
# y = [1 if "yes" in x else 0 for x in X.to_numpy()[:,0:1]]
# P = Perceptron(.05, 5).fit(X.iloc[:40,1:], y)
# P.predict(X.iloc[40:,1:])

    year  temp  humidity  rainfall  drought_code  buildup_index  day  month  \
0   2015    28        59       0.0          8.06           3.47    1      6   
1   2010    30        61       1.3          8.17           4.03    2      6   
2   2009    26        83      13.1          8.08           3.59    3      6   
3   2017    25        87       2.5          7.18           2.42    4      6   
4   2014    28        77       0.0         14.98           4.63    5      6   
5   2008    30        67       0.0         22.71           7.97    6      6   
6   2013    32        52       0.0         30.98          11.86    7      6   
7   2009    29        72       0.0         38.66          14.31    8      6   
8   2014    26        89       0.2         39.62          10.76    9      6   
9   2007    28        77       0.0         46.44          13.14   10      6   
10  2015    30        67       0.0         54.44          16.10   11      6   
11  2009    26        81       0.0         61.80    

ValueError: shapes (164,9) and (8,) not aligned: 9 (dim 1) != 8 (dim 0)